In [7]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup


In [3]:
df=pd.read_json('movies_with_imdbID.json')


In [4]:
df.head()

,Title,Rating,Metadata,Info,Director,Cast,Genres,Image,Trailer,imdbID
0,The Shawshank Redemption,"[9.3, (2.9M)]","[1994, 2h 22m, R]","Over the course of several years, two convicts...",[Frank Darabont],"[Tim Robbins, Morgan Freeman, Bob Gunton]","[Action, Crime, Drama]",{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=PLl99DlL6b4,tt0111161
1,The Godfather,"[9.2, (2M)]","[1972, 2h 55m, R]",The aging patriarch of an organized crime dyna...,[Francis Ford Coppola],"[Marlon Brando, Al Pacino, James Caan]","[Crime, Drama]",{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=UaVTIH8mujA,tt0068646
2,The Dark Knight,"[9.0, (2.9M)]","[2008, 2h 32m, PG-13]",When the menace known as the Joker wreaks havo...,[Christopher Nolan],"[Christian Bale, Heath Ledger, Aaron Eckhart]","[Crime, Drama]",{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=EXeTwQWrcwY,tt0468569
3,The Godfather Part II,"[9.0, (1.4M)]","[1974, 3h 22m, R]",The early life and career of Vito Corleone in ...,[Francis Ford Coppola],"[Al Pacino, Robert De Niro, Robert Duvall]",[Drama],{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=9O1Iy9od7-A,tt0071562
4,12 Angry Men,"[9.0, (861K)]","[1957, 1h 36m, Approved]",The jury in a New York City murder trial is fr...,[Sidney Lumet],"[Henry Fonda, Lee J. Cobb, Martin Balsam]","[Crime, Drama]",{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=TEN-2uTi2c0,tt0050083


In [10]:
HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

r = requests.get('https://www.imdb.com/title/tt0111161/', headers=HEADERS)
soup = BeautifulSoup(r.text, 'html.parser')


In [11]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <script>
   if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }
  </script>
  <script>
   window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1713069323319);
        }
    })
  </script>
  <title>
   The Shawshank Redemption (1994) - IMDb
  </title>
  <meta content="The Shawshank Redemption: Directed by Frank Darabont. With Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler. Over the course of several years, two convicts

In [13]:
# get this <script type="application/ld+json"> and get the json object
json_object = json.loads(soup.find('script', type='application/ld+json').string)
print(json_object['genre'])


['Drama']


In [15]:
def get_genre(imdbID):
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

    for attempt in range(2):  # Try twice
        try:
            r = requests.get(f'https://www.imdb.com/title/{imdbID}/', headers=HEADERS)
            soup = BeautifulSoup(r.text, 'html.parser')
            json_object = json.loads(soup.find('script', type='application/ld+json').string)
            return json_object['genre']
        except Exception as e:
            if attempt < 1:  # If this was the first attempt, continue to the next attempt
                continue
            else:  # If this was the second attempt, return an error message
                return f'Error getting genre for {imdbID}: {e}'

In [16]:
for index, row in df.iterrows():
    df.at[index, 'genre'] = get_genre(row['imdbID'])
    print(f"Got genre for {row['imdbID']}")

Got genre for tt0111161
Got genre for tt0068646
Got genre for tt0468569
Got genre for tt0071562
Got genre for tt0050083
Got genre for tt0108052
Got genre for tt0167260
Got genre for tt0110912
Got genre for tt0120737
Got genre for tt0060196
Got genre for tt0109830
Got genre for tt0167261
Got genre for tt0137523
Got genre for tt1375666
Got genre for tt15239678
Got genre for tt0080684
Got genre for tt0133093
Got genre for tt0099685
Got genre for tt0073486
Got genre for tt0114369
Got genre for tt0816692
Got genre for tt0038650
Got genre for tt0047478
Got genre for tt0102926
Got genre for tt0120815
Got genre for tt0317248
Got genre for tt0118799
Got genre for tt0120689
Got genre for tt0103064
Got genre for tt0076759
Got genre for tt0088763
Got genre for tt0245429
Got genre for tt0253474
Got genre for tt6751668
Got genre for tt0054215
Got genre for tt9362722
Got genre for tt0172495
Got genre for tt0110357
Got genre for tt0110413
Got genre for tt0407887
Got genre for tt0120586
Got genre for t

In [17]:
df.head()

,Title,Rating,Metadata,Info,Director,Cast,Genres,Image,Trailer,imdbID,genre
0,The Shawshank Redemption,"[9.3, (2.9M)]","[1994, 2h 22m, R]","Over the course of several years, two convicts...",[Frank Darabont],"[Tim Robbins, Morgan Freeman, Bob Gunton]","[Action, Crime, Drama]",{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=PLl99DlL6b4,tt0111161,[Drama]
1,The Godfather,"[9.2, (2M)]","[1972, 2h 55m, R]",The aging patriarch of an organized crime dyna...,[Francis Ford Coppola],"[Marlon Brando, Al Pacino, James Caan]","[Crime, Drama]",{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=UaVTIH8mujA,tt0068646,"[Crime, Drama]"
2,The Dark Knight,"[9.0, (2.9M)]","[2008, 2h 32m, PG-13]",When the menace known as the Joker wreaks havo...,[Christopher Nolan],"[Christian Bale, Heath Ledger, Aaron Eckhart]","[Crime, Drama]",{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=EXeTwQWrcwY,tt0468569,"[Action, Crime, Drama]"
3,The Godfather Part II,"[9.0, (1.4M)]","[1974, 3h 22m, R]",The early life and career of Vito Corleone in ...,[Francis Ford Coppola],"[Al Pacino, Robert De Niro, Robert Duvall]",[Drama],{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=9O1Iy9od7-A,tt0071562,"[Crime, Drama]"
4,12 Angry Men,"[9.0, (861K)]","[1957, 1h 36m, Approved]",The jury in a New York City murder trial is fr...,[Sidney Lumet],"[Henry Fonda, Lee J. Cobb, Martin Balsam]","[Crime, Drama]",{'140w': 'https://m.media-amazon.com/images/M/...,https://www.youtube.com/watch?v=TEN-2uTi2c0,tt0050083,"[Crime, Drama]"


In [18]:
# save the dataframe to a json file
df.to_json('movies_with_imdbID.json', orient='records')

In [20]:
# remove the Genres column
df.drop(columns=['Genres'], inplace=True)

In [21]:
# save this to overwrite the previous file
df.to_json('movies_with_imdbID.json', orient='records')